In [121]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Descarga de pacientes y empresas

In [122]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

(7886, 11)

In [123]:
# recuperar todos los pacientes ID desdde aAccount
results = sf.query_all(
    """
  select Id, IdentificationId__pc,Name,poliza__c, Identification_Type__c, IsPersonAccount, Ocupaci_n__c, Pa_s_del_Paciente__c, PersonTitle ,PersonDepartment  from Account where ispersonaccount = True
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name",'poliza__c','Identification_Type__c','IsPersonAccount','Ocupaci_n__c','Pa_s_del_Paciente__c','PersonTitle','PersonDepartment']]
Pacientes_SF.shape

(1534724, 10)

In [124]:
# recuperar todos los pacientes ID desde Contact
results = sf.query_all(
    """
   select Id,Name,AccountId  from Contact
    """
)
Pacientesc_SF = pd.DataFrame(results["records"])

Pacientesc_SF = Pacientesc_SF[["Id", "AccountId", "Name"]]
Pacientesc_SF.shape

(1534724, 3)

In [125]:
#Relacion AccountContactRelacion se descarga la relacion paciente Empresa en account contact no es la q se ve
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

(1222508, 5)

In [126]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name
0,07k8c00000RGWPNAA5,0018c00002axQa5AAE,0038c000036jLDrAAM,Metlife Holding Metlife Finning,174365172,0038c000036jLDrAAM,0018c00002cVCmPAAW,Correa Aguilar Rene Alberto
1,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas
2,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban
3,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen
4,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres


In [127]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId_x",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id,Name_y,IdentificationId__pc,Identification_Type__c,Holding__c,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c
0,07k8c00000RGWPNAA5,0018c00002axQa5AAE,0038c000036jLDrAAM,Metlife Holding Metlife Finning,174365172,0038c000036jLDrAAM,0018c00002cVCmPAAW,Correa Aguilar Rene Alberto,0018c00002axQa5AAE,Metlife Holding Metlife Finning,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5951,None,None
1,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
2,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
3,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
4,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None


In [128]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19232\3766479298.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Id_x'} in the result is deprecated and will raise a MergeError in a future version.
  Union_account_contact_relation = pd.merge(


,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id_x,Name_y,...,Id_y,IdentificationId__pc_y,Name,poliza__c_y,Identification_Type__c_y,IsPersonAccount,Ocupaci_n__c,Pa_s_del_Paciente__c_y,PersonTitle,PersonDepartment
0,07k8c00000RGWPNAA5,0018c00002axQa5AAE,0038c000036jLDrAAM,Metlife Holding Metlife Finning,174365172,0038c000036jLDrAAM,0018c00002cVCmPAAW,Correa Aguilar Rene Alberto,0018c00002axQa5AAE,Metlife Holding Metlife Finning,...,0018c00002cVCmPAAW,174365172,Correa Aguilar Rene Alberto,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
1,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWElAAO,182204846,Maldonado Velasquez Nicolas,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
2,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amMfoAAE,18396206K,Zaror Yunis Daniel Esteban,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
3,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amRHeAAM,184262452,Bustamante Morales Flor Del Carmen,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
4,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWEmAAO,184429772,Malleo Abarca Rigoberto Andres,None,RUT,True,None,a1U8c000005dAtnEAE,None,None


In [129]:
#buscar empresas desde Programas_por_Empresas__c
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7892, 5)

In [130]:
#buscar empresas desde Paciente_Programas__c
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1222351, 4)

In [131]:
#unir empresas y todas sus caracteristicas


empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
empresas_unidas.shape

(1222351, 9)

In [132]:
empresas_unidas.head()
apprix_1 = empresas_unidas.iloc[:611178,:]
apprix_2 = empresas_unidas.iloc[611178:,:]
apprix_1.to_excel(
    "C:/Users/jmartinez/Documents/empresasA.xlsx", index=False
)
apprix_2.to_excel(
    "C:/Users/jmartinez/Documents/empresasB.xlsx", index=False
)

Descarga desde Citas de Servicios

In [133]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,ParentRecordId,Tipo_de_la_cita__c from ServiceAppointment
    """
)
Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",
        "RUT__c",
        "CreatedDate",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "AccountId",
        "WorkTypeId" 
    ]
]
Citas_de_servicio.shape


(6896, 7)

In [134]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c,RecordTypeId from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])
ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_deporte.shape


(689, 8)

In [ ]:
#Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])
ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_nutricion.shape

In [ ]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])
ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Apssex.shape

In [ ]:
#Clinica Psicologia
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])
ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Psicologia.shape

In [ ]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Sueno.shape

In [ ]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Cronico.shape

In [ ]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Telemedicina.shape

In [ ]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Veterinaria.shape

In [ ]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion['RecordTypeId'] = 'Ficha de Derivacion'
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Orientacion.shape

In [135]:
Citas_de_servicio.head()

,AppointmentNumber,RUT__c,CreatedDate,CreatedDate,Tipo_de_la_cita__c,AccountId,WorkTypeId
0,SA-3124,183552252,2022-12-02T16:23:05.000+0000,2022-12-02T16:23:05.000+0000,Phone,0018c00002azpJ4AAI,08q8c000000VW1yAAG
1,SA-4987,224488556,2022-12-21T21:28:49.000+0000,2022-12-21T21:28:49.000+0000,Phone,0018c00002amLbpAAE,08q8c000000VW1yAAG
2,SA-3905,185384799,2022-12-12T15:34:28.000+0000,2022-12-12T15:34:28.000+0000,Phone,0018c00002ay6KFAAY,08q8c000000VW24AAG
3,SA-3871,187821444,2022-12-12T12:21:02.000+0000,2022-12-12T12:21:02.000+0000,Phone,0018c00002bI5vJAAS,08q8c000000VVsBAAW
4,SA-3898,167466923,2022-12-12T15:04:05.000+0000,2022-12-12T15:04:05.000+0000,Phone,0018c00002ao4fCAAQ,08q8c000000VVs1AAG


In [136]:
Citas_de_servicio.dtypes

AppointmentNumber     object
RUT__c                object
CreatedDate           object
CreatedDate           object
Tipo_de_la_cita__c    object
AccountId             object
WorkTypeId            object
dtype: object

In [137]:
Citas_de_servicio['Mes'] =  pd.to_datetime(Citas_de_servicio['CreatedDate']).dt.month 
Citas_de_servicio['Mes'] =  pd.to_datetime(Citas_de_servicio['CreatedDate']).dt.month 
Citas_de_servicio['Año'] =  pd.to_datetime(Citas_de_servicio['CreatedDate']).dt.year
Citas_de_servicio['Dia'] =  pd.to_datetime(Citas_de_servicio['CreatedDate']).dt.day

#x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Citas_de_servicio["Fecha"] = pd.to_datetime(Citas_de_servicio['CreatedDate'], dayfirst=True,yearfirst=False, exact=True)
Citas_de_servicio['Fecha3'] = Citas_de_servicio['Fecha'].dt.strftime('%d-%m-%Y')
Citas_de_servicio.head()


ValueError: cannot assemble with duplicate keys

In [ ]:

Citas_de_servicio['Fecha'] = Citas_de_servicio['Fecha'].dt.tz_localize(None)

Citas_de_servicio.to_excel(
    "C:/Users/jmartinez/Documents/fecha1.xlsx", index=False
)
Citas_de_servicio.dtypes



In [ ]:
Citas_de_servicio['Fecha3'] = pd.to_datetime(Citas_de_servicio['Fecha3'],format="%d-%m-%Y")
Citas_de_servicio.head()

In [ ]:
Citas_de_servicio.dtypes


In [ ]:
Citas_de_servicio.head()

In [ ]:
Citas_de_servicio['Fecha'] = Citas_de_servicio['Fecha'].dt.tz_localize(None)
Citas_de_servicio.to_excel(
    "C:/Users/jmartinez/Documents/fecha2.xlsx", index=False
)

In [ ]:
Citas_de_servicio['Meses'] = (pd.to_datetime(Citas_de_servicio['CreatedDate'], format='%Y.%m.%d', errors="coerce")
               .dt.month_name(locale='es_ES.utf8')
               )

In [ ]:
Citas_de_servicio.head()
